In [ ]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10

In [ ]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol without an attacker.



In [ ]:
"""As usual, Alice and Bob want to establish a shared key consisting of a
sequence of N bits. They rely on a seqence of entangled pairs of qubits,
which can be sent to them by a third party. Each entangled pair is in the
state 1/√2(|01⟩ − |10⟩)."""

from qiskit_aer import Aer
from qiskit import transpile, assemble

circuit = QuantumCircuit(2,2) 
circuit.h(0) 
circuit.cx(0,1)
circuit.z(1)
circuit.measure(range(2),range(2))
backend = BasicSimulator()
compiled = transpile(circuit, backend)
job_sim = backend.run(compiled, shots=1024)
result_sim = job_sim.result() 
counts = result_sim.get_counts(compiled)
print(counts)
circuit.draw("mpl")

In [ ]:
"""We saw in Lecture 8 that Alice and Bob can test the entanglement of the
sequence of pairs of qubits. To do this, for each pair, Alice randomly chooses
to measure Z or X, and Bob randomly chooses to measure W = 1/√2(X + Z) or V = 1/√2(X − Z)."""

import random

a_choice = random.choice(['X','Z'])
b_choice = random.choice(['W','V'])

c = circuit.copy()

if a_choice == 'X':
    c.h(0)
    
c.h(1)
c.z(1)

if b_choice == 'V':
    c.y(1)

c.measure(range(2), range(2))
c.draw('mpl')

In [ ]:
"""To perform the entanglement test, Alice and Bob do a series of measurements on the
sequence of pairs of qubits. They calculate
                S = |⟨X ⊗ W⟩ − ⟨X ⊗ V ⟩ + ⟨Z ⊗ W⟩ + ⟨Z ⊗ V ⟩|.
What does this mean? ⟨X ⊗ W⟩ is the average value of the result of measuring X ⊗ W.
Measuring X ⊗ W means measuring Alice’s qubit in the basis corresponding to operator
X and measuring Bob’s qubit in the basis corresponding to operator W.
The results of these measurements are converted from {0, 1} to {+1, −1} and multiplied
together. The average value is calculated by adding up these results across all the
pairs of qubits, and dividing by the length of the sequence. For each measurement, Alice
and Bob make independent random choices of measurement, and the results are added to the
totals for ⟨X ⊗ W⟩, ⟨X ⊗ V ⟩, ⟨Z ⊗ W⟩ or ⟨Z ⊗ V ⟩ as appropriate. Finally take the
absolute value. If the value is close to 2√2 then there is entanglement, but if the value
is below 2 then there are only classical correlations. This is all in Lecture 8 and Lab 4B."""

from qiskit_aer import Aer
import numpy as np

def measure_entanglement(n_shots=1024):
    results = {"XW": [], "XV": [], "ZW": [], "ZV": []}
    
    for _ in range(n_shots):
        circuit = QuantumCircuit(2, 2)
        circuit.h(0)
        circuit.cx(0, 1)
        circuit.z(1)
        
        a_choice = random.choice(["X", "Z"])
        b_choice = random.choice(["W", "V"]) 

        if a_choice == "X":
            circuit.h(0)
        
        circuit.h(1)
        circuit.z(1)
        
        if b_choice == "V":
            circuit.y(1)
        
        circuit.measure(range(2), range(2))
        
        backend = Aer.get_backend("qasm_simulator")
        job = transpile(circuit, backend)
        job_sim = backend.run(job, shots=1)
        result_sim = job_sim.result() 
        counts = result_sim.get_counts(job)
        
        outcome = list(counts.keys())[0]
        a_result = (-1) ** int(outcome[1])
        b_result = (-1) ** int(outcome[0])
        product = a_result * b_result
        
        key = a_choice + b_choice
        results[key].append(product)
    
    expectation_values = {k: np.mean(v) for k, v in results.items()}
    
    S = abs(expectation_values["XW"] - expectation_values["XV"] + expectation_values["ZW"] + expectation_values["ZV"])
    
    return S

s_value = measure_entanglement()
print(f"S = {s_value}")
if s_value > 2:
    print("Entanglement detected!")
else:
    print("Only classical correlations.")

In [ ]:
"""Alice and Bob each work with three operators that they can measure.
Alice’s operators are
A1 = X, A2 = W, A3 = Z
Bob’s operators are
B1 = W, B2 = Z, B3 = V"""

a_operators = ['X','W','Z']
b_operators = ['W','Z','V']

In [ ]:
"""
Now for the protocol itself. Alice and Bob are trying to establish a shared
key of length N. They go through the following steps 9N/2 times, storing the results
of their measurements and their choices of basis.

1. Alice and Bob each receive one qubit of an entangled pair in state 1/√2(|01⟩ − |10⟩).
2. Alice chooses an operator Ai randomly from her set of three, with probability 1/3 each.
3. Bob chooses an operator Bj randomly from his set of three, with probability 1/3each.
4. Alice measures operator Ai on her qubit. The details of how to do this are in Lab 4B.
5. Bob measures operator Bj on his qubit, again as in Lab 4B.
"""

key_length = 10

shared_key = []
entanglement_data = {"XW": [], "XV": [], "ZW": [], "ZV": []}

for i in range(math.ceil((9*key_length)/2)):
    c = circuit.copy()

    a_num = random.randint(0,2)
    b_num = random.randint(0,2)
    
    a_op = a_operators[a_num]
    b_op = b_operators[b_num]

    if a_op == "X":
        c.h(0)
    elif a_op == "W":
        c.z(0)
    
    if b_op == "W" or b_op == "V":
        c.h(1)
        c.z(1)
        if b_op == "V":
            c.y(1)

    c.measure(range(2), range(2))    

    transpiled_circuit = transpile(c, backend)
    job_sim = backend.run(transpiled_circuit, shots=1)
    result_sim = job_sim.result()
    counts = result_sim.get_counts()

    outcome = list(counts.keys())[0]
    a_result = int(outcome[1])  # Alice's bit (qubit 0)
    b_result = int(outcome[0])  # Bob's bit (qubit 1)

    # Convert {0,1} to {+1,-1} for S calculation
    a_value = (-1) ** a_result
    b_value = (-1) ** b_result
    product = a_value * b_value

    if (a_op == "Z" and b_op == "W") or (a_op == "W" and b_op == "V"):
        shared_key.append(a_result)
    else:
        # Store entanglement verification data
        key = a_choice + b_choice
        entanglement_data[key].append(product)
            
expectation_values = {k: np.mean(v) if v else 0 for k, v in entanglement_data.items()}
S = abs(expectation_values["XW"] - expectation_values["XV"] + expectation_values["ZW"] + expectation_values["ZV"])

# Bob inverts his key bits (since Alice and Bob's results are opposite)
final_key = [1 - bit for bit in shared_key]  # Flip 0 ↔ 1

# Print results
print(f"Final Shared Key: {''.join(map(str, final_key))} (Length: {len(final_key)})")
print(f"S = {S}")
if S > 2:
    print("Entangled.")
else:
    print("Attacker present.")